In [17]:
# Load acts and toks

import torch
import numpy as np
from transformers import AutoTokenizer
from interp_utils import reload_module
from sparse_models import SimpleSparseMLP
from feature_kit import *
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

fname = f'mlp_F6000_clipped'

acts = torch.load(f'cached_acts/train/{fname}_sparse.pt')

# acts = [acts[i].to_dense()[:300].to_sparse() for i in range(len(acts))]
# acts[1000] = acts[1000].to_dense()[:3000]
# per_doc_maxes = acts.max(dim=-1).values
# per_doc_maxes = per_doc_maxes/(per_doc_maxes.max(dim=-1).values[:,None]+1)

docs = torch.load('cached_acts/train/train_tok_strs.pt')[:3000]
docs = np.array([[tok.replace(' ', '○').replace('\n', '↵') for tok in doc] for doc in docs])
doc_ids = torch.load('cached_acts/train/train_tok_ids.pt')[:3000]

mlp = SimpleSparseMLP(n_features=6000, d_model=768)
mlp.load_state_dict(torch.load(f'./sparse-mlps/{fname}.pt', map_location=device))

print(fname)

mlp_F6000_clipped


In [18]:
# doc_strings = [''.join(doc[:128]) for doc in docs.tolist()]

# def filter_docs(and_=[], or_=[], not_=[]):
#     '''
#     and_: list of strings
#     or_: list of strings
#     not_: list of strings

#     filter docs for documents that include every string in and_ as a substring, at least one substring in or_, and no substrings in not_
#     '''
#     return np.array([i for i, doc in enumerate(doc_strings) if (any([inc in doc for inc in or_]) or len(or_) == 0) and (not any([exc in doc for exc in not_]) or len(not_) == 0) and all([inc in doc for inc in and_])])

# def get_feature_data(feature_idx, and_=[], or_=[], not_=[], reversed=False):
#     subset = filter_docs(and_=and_, or_=or_, not_=not_)
#     print(f'Found {len(subset)} docs')

#     feature_acts = acts[feature_idx].to_dense()
#     feature_acts = ((feature_acts/feature_acts.max())[subset])

#     perm = feature_acts.max(dim=-1).values.argsort(descending=reversed)
#     feature_acts = feature_acts[perm]
#     feature_docs = docs[subset][perm]
#     feature_doc_ids = doc_ids[subset][perm]
#     per_doc_feature_maxes = feature_acts.max(dim=-1).values

#     return feature_docs, feature_doc_ids, feature_acts, per_doc_feature_maxes

In [21]:
pysvelte.__file__

'/home/noa/PySvelte/pysvelte/__init__.py'

In [31]:
feature_acts.max()

tensor(1.)

In [2]:
from interp_utils import reload_module
reload_module('pysvelte')
import pysvelte

pysvelte.WeightedDocsControl().publish('./test.html')



pysvelte components appear to be unbuilt or stale
Building pysvelte components with webpack...

> pysvelte@1.0.0 webpack
> node ./node_modules/.bin/webpack

entry: {"loader":"./src/loader.js","AttentionMulti":"./src/AttentionMulti.svelte","DoubleSlider":"./src/DoubleSlider.svelte","Hello":"./src/Hello.svelte","TopKTable":"./src/TopKTable.svelte","WeightedDoc":"./src/WeightedDoc.svelte","WeightedDocs":"./src/WeightedDocs.svelte","WeightedDocsControl":"./src/WeightedDocsControl.svelte"}
asset AttentionMulti.js 47.5 KiB [emitted] [minimized] (name: AttentionMulti)
asset TopKTable.js 39.7 KiB [emitted] [minimized] (name: TopKTable)
asset WeightedDocsControl.js 37.5 KiB [emitted] [minimized] (name: WeightedDocsControl)
asset DoubleSlider.js 34.8 KiB [emitted] [minimized] (name: DoubleSlider)
asset WeightedDocs.js 33.5 KiB [emitted] [minimized] (name: WeightedDocs)
asset loader.js 32.2 KiB [emitted] [minimized] (name: loader)
asset WeightedDoc.js 29.9 KiB [emitted] [minimized] (name: Weighte

'./test.html'

In [2]:
from interp_utils import reload_module
reload_module('pysvelte')
import pysvelte

# FEATURE_IDX = 1500
# # feature_docs, feature_doc_ids, feature_acts, feature_maxes = get_feature_data(feature_idx=FEATURE_IDX, and_=[], or_=[], not_=[], reversed=False)
# # print(FEATURE_IDX)

# feature_acts = acts[FEATURE_IDX].to_dense()
# feature_acts = feature_acts/feature_acts.max()

# pysvelte.WeightedDocs(tokens=docs.tolist(), weights=feature_acts).show()#.publish('./test.html')#, per_doc_maxes=feature_maxes.tolist()).show()

pysvelte components appear to be unbuilt or stale
Building pysvelte components with webpack...

> pysvelte@1.0.0 webpack
> node ./node_modules/.bin/webpack

entry: {"loader":"./src/loader.js","AttentionMulti":"./src/AttentionMulti.svelte","Hello":"./src/Hello.svelte","Test":"./src/Test.svelte","TopKTable":"./src/TopKTable.svelte","WeightedDoc":"./src/WeightedDoc.svelte","WeightedDocs":"./src/WeightedDocs.svelte"}


KeyboardInterrupt: 

In [ ]:
reload_module('feature_kit')
from feature_kit import Seq
hypothesis = Seq(' He', ' looked', ' down')
hypothesis

In [ ]:
def feature_filter(docs, weights, feature):
    feat_indices = feature.get_feature(doc_ids).max(dim=-1).values.nonzero().flatten()
    print(feat_indices)
    print(docs.shape)
    return docs[feat_indices], weights[feat_indices]
    


In [ ]:
hypothesis.feats[0](enc(' a', ' test', '.', ' Every')[None].cuda())


In [ ]:
period_every.get_feature(enc('.', ' Every')[None].cuda())

In [ ]:
from interp_utils import reload_module
reload_module('feature_kit')
from feature_kit import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

# hypothesis = Match(' to')
hypothesis = Sum(
    ' Every',
    'Every',
    ' Each',
    ' every',
    ' each',
    ' Everyday',
).cuda()
period_every = Seq('.', ' Every').cuda()
feature_acts = feature_acts.cuda()
doc_ids = doc_ids.cuda()

opt = optim.Adam(hypothesis.parameters(), lr=1e-2)
pbar = tqdm(range(300))
for epoch_index in pbar:
    loss = ((hypothesis(feature_doc_ids) - feature_acts)**2).mean()
    pbar.set_description(f'Loss: {loss:.2E}')
    loss.backward()
    opt.step()
    opt.zero_grad()
    
hypothesis_acts = hypothesis(feature_doc_ids)
errors = ((hypothesis_acts-feature_acts)**2).abs()
# neg_errors = (feature_acts-hypothesis_acts).clamp(min=0)
# perm = errors.min(dim=-1).values.argsort(descending=True)
# # # perm = torch.abs(hypothesis(doc_ids) - feature_acts).max(dim=0).values.sort(descending=False).indices
# # # perm = errors.min(dim=-1).values.argsort(descending=False)
# # errors = errors.detach()[perm] #?
# # per_doc_maxes = errors.max(dim=-1).values
# pysvelte.WeightedDocs(tokens=feature_docs[perm].tolist(), weights=hypothesis_acts[perm].tolist(), per_doc_maxes=feature_maxes[perm].tolist()).show()

# tokens, weights = feature_filter(feature_docs, hypothesis.get_feature(acts), feature=hypothesis)
print(hypothesis)
print(f'vs baseline: {loss/(feature_acts**2).mean()}')
pysvelte.WeightedDocs(tokens=feature_docs, weights=errors.tolist(), reversed=False)#.publish('./test.html')


In [ ]:
feature_docs[0]

In [ ]:
hypothesis

In [ ]:


import pysvelte
reload_module('tok_labelling')
from tok_labelling import new_tok_label, load_tok_label

# new_tok_label(label='water_words', description="Strings that are related to water. For example, 'water', 'bath', 'overflow', 'tank', 'wet', 'ducks'")
water_words = load_tok_label('water_words')

water_word_match = Match(*water_words)
# water_word_cases = Seq(
#     Cases(*water_words)
# )

feats_fn = Stack(
    Match(' to')
)

MAX_DOCS_TO_PYSVELTE = 100000

FIND_HOLES = False
HIDE_POS_ERRS = False
USE_MSE_IN_RENDERER = False

feature_acts = acts[FEATURE_IDX].to_dense()
feature_acts = feature_acts/feature_acts.max()
reg_weights, reg_bias, pred = pred_feature(feature_acts, feats_fn, doc_ids)
print(reg_weights)

mse_errs = (feature_acts-pred)**2
abs_errs = (feature_acts-pred).abs()

signed_errs = mse_errs*(feature_acts-pred).sign() if USE_MSE_IN_RENDERER else abs_errs*(feature_acts-pred).sign()
doc_maxes = mse_errs.max(dim=-1).values
err_docs = doc_maxes.topk(k=10).indices

# sort docs by error
if FIND_HOLES:
    perm = signed_errs.min(dim=-1).values.clamp(max=0).abs().argsort(descending=False)[-MAX_DOCS_TO_PYSVELTE:]
    signed_errs = signed_errs[perm]
else:
    perm = mse_errs.max(dim=-1).values.argsort(descending=False)[-MAX_DOCS_TO_PYSVELTE:]
    signed_errs = signed_errs[perm]

if HIDE_POS_ERRS:
    signed_errs = signed_errs.clamp(max=0)


mse = mse_errs.mean()
print(f'mse: {mse:.2E}')
abs_errs = abs_errs[perm]
feature_doc_subset = docs[perm] 
# weights = feature_acts[perm]s

# feature_docs
component = pysvelte.WeightedDocs(tokens=feature_doc_subset.tolist(), weights=signed_errs.tolist(), per_doc_maxes=doc_maxes.tolist(), reversed=False)
component.show()

